In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

DATA_DIR = "data_pre"
IMG_HEIGHT = 128
IMG_WIDTH = 128
CHANNELS = 3
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATA_DIR}/train",
    labels="inferred",
    label_mode="binary",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATA_DIR}/val",
    labels="inferred",
    label_mode="binary",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATA_DIR}/test",
    labels="inferred",
    label_mode="binary",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False
)

for images, labels in train_ds.take(1):
    print(labels.numpy())

2025-06-02 15:34:40.806103: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-02 15:34:40.809587: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-02 15:34:40.818947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748853280.833814    2644 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748853280.838285    2644 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748853280.851073    2644 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Found 22896 files belonging to 2 classes.
Found 2514 files belonging to 2 classes.


E0000 00:00:1748853284.186332    2644 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1748853284.188843    2644 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 6206 files belonging to 2 classes.
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]]


2025-06-02 15:34:44.506952: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs available:")
    for gpu in gpus:
        print(f"  {gpu}")
else:
    print("No GPUs found. TensorFlow will run on CPU.")

GPUs available:
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
import jax

# --- PRIMARY CHECK: This is the most reliable way ---
print("--- JAX Device Information ---")
devices = jax.devices()
print("JAX detected devices:")
for device in devices:
    print(f"  {device}")

gpu_devices = [d for d in devices if d.device_kind == 'gpu']
if gpu_devices:
    print(f"\nSUCCESS: JAX is configured to use GPU backend. Found {len(gpu_devices)} GPU(s).")
    print(f"JAX default backend: {jax.default_backend()}")
else:
    print("\nWARNING: No GPU devices found. JAX is likely running on CPU.")
    print(f"JAX default backend: {jax.default_backend()}")

# --- Alternative/Internal Check (if you're curious, but less critical) ---
# The previous error indicates 'get_backend()' moved or was removed from xla_extension
# The correct way to get the backend through the bridge is:
try:
    print(f"\nJAX XLA bridge platform: {jax.lib.xla_bridge.get_backend().platform}")
except Exception as e:
    print(f"Error accessing jax.lib.xla_bridge.get_backend().platform: {e}")

print(f"\nJAX version: {jax.__version__}")
print(f"jaxlib version: {jax.lib.__version__}")

# Check for CUDA/cuDNN version info again (still might not be available directly)
try:
    # This path depends on jaxlib's internal structure and can change.
    # It might be in jax.lib.xla_client or jax._src.lib.xla_client
    # Let's try a common path for newer jaxlib versions.
    if hasattr(jax.lib, 'xla_client') and hasattr(jax.lib.xla_client, 'cuda_runtime_version'):
        print(f"Built with CUDA (runtime): {jax.lib.xla_client.cuda_runtime_version()}")
    else:
        print("CUDA runtime version info not directly available through jax.lib.xla_client.")

    if hasattr(jax.lib, 'xla_client') and hasattr(jax.lib.xla_client, 'cudnn_version'):
        print(f"Built with cuDNN: {jax.lib.xla_client.cudnn_version()}")
    else:
        print("cuDNN version info not directly available through jax.lib.xla_client.")

except AttributeError:
    print("CUDA/cuDNN version info not directly available (AttributeError).")
except Exception as e:
    print(f"Error retrieving CUDA/cuDNN info (general): {e}")

--- JAX Device Information ---
JAX detected devices:
  TFRT_CPU_0

JAX default backend: cpu

JAX XLA bridge platform: cpu

JAX version: 0.4.28
jaxlib version: 0.4.28
CUDA runtime version info not directly available through jax.lib.xla_client.
cuDNN version info not directly available through jax.lib.xla_client.
